In [1]:
# Задача 1 Удвоение чисел и получение первого результата
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# Загрузка списка из файла
def load_numbers(file_path):
    with open(file_path, 'r') as file:
        data = file.readlines()
    return [list(map(int, line.strip().split(','))) for line in data]

# Обработка числа: удвоение и задержка
def process_number(number):
    time.sleep(0.2)  # Имитируем задержку
    return number * 2

# Основная функция обработки списков
file_path = 'D:/Эстер/с рс/Courses/ML Engineer/HWEster7/test_list_numbers.txt'
numbers_list = load_numbers(file_path)

results = []  # Список для хранения результатов

# Инициализация ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_list = {
        executor.submit(lambda nums: [process_number(n) for n in nums], num_list): idx
        for idx, num_list in enumerate(numbers_list)
    }

    completed_lists = []  # Список для отслеживания завершенных списков

    for future in as_completed(future_to_list):
        list_index = future_to_list[future]
        try:
            result = future.result()
            completed_lists.append((list_index, result))
        except Exception as exc:
            print(f"Список {list_index} вызвал ошибку: {exc}")

# Поиск первого обработанного списка
if completed_lists:
    first_processed_list = completed_lists[0][1]  # Результаты первого завершенного списка
    first_list_sum = sum(first_processed_list)
    print(f"Сумма чисел в первом обработанном списке: {first_list_sum}")
else:
    print("Не удалось обработать списки.")


Сумма чисел в первом обработанном списке: 11090


In [ ]:
# Задача 2 Поиск и суммирование чисел через цепочку файлов
import os
import zipfile
from multiprocessing import Pool
import tempfile

# Путь к архивам
base_path = "D:/Эстер/с рс/Courses/ML Engineer/HWEster7/"
archive1 = os.path.join(base_path, "path_8_8.zip")
archive2 = os.path.join(base_path, "recursive_challenge_8_8.zip")

# Временные директории для извлечения архивов
temp_dir1 = tempfile.mkdtemp()
temp_dir2 = tempfile.mkdtemp()

# Извлечение архивов
with zipfile.ZipFile(archive1, 'r') as zip1:
    zip1.extractall(temp_dir1)

with zipfile.ZipFile(archive2, 'r') as zip2:
    zip2.extractall(temp_dir2)

# Обработка одного файла из первого архива
def process_file(file_name):
    # Шаг 1: Прочитать путь из файла в temp_dir1
    file_path = os.path.join(temp_dir1, file_name)
    with open(file_path, 'r') as file:
        next_path = file.read().strip()
    
    # Шаг 2: Прочитать число из конечного файла в temp_dir2
    final_file_path = os.path.join(temp_dir2, next_path)
    with open(final_file_path, 'r') as file:
        return int(file.read().strip())

# Главная функция
if __name__ == "__main__":
    # Список файлов из первого архива
    file_list = os.listdir(temp_dir1)
    
    # Обработка файлов в пуле процессов
    with Pool() as pool:
        results = pool.map(process_file, file_list)

    # Суммировать все числа
    total_sum = sum(results)
    print(f"Сумма всех чисел: {total_sum}")
